<a href="https://colab.research.google.com/github/harneet862/FireWatch/blob/main/Generatingno_firedata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry
import time
from datetime import datetime
from datetime import timedelta

In [ ]:
df_dropped = pd.read_csv('/content/filename (1).csv')

<ipython-input-5-5140dbd2449f>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dropped = pd.read_csv('/content/filename (1).csv')


In [ ]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
batch_size = 300
batch_start = 14978
while batch_start <= 16000:
    for i in range(batch_size):
        date = datetime.strptime(df_dropped['reported_date'].iloc[i + batch_start], "%Y-%m-%d %H:%M:%S").date()
        hour = datetime.strptime(df_dropped['reported_date'].iloc[i + batch_start], "%Y-%m-%d %H:%M:%S").time()
        params = {
            "latitude": df_dropped.at[i, 'fire_location_latitude'],
            "longitude": df_dropped.at[i, 'fire_location_longitude'],
            "start_date": date - timedelta(days=1),
            "end_date": date,
            "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", "rain", "snowfall", "snow_depth", "weather_code", "cloud_cover", "et0_fao_evapotranspiration", "vapour_pressure_deficit", "wind_speed_100m", "wind_direction_10m", "wind_gusts_10m", "soil_temperature_0_to_7cm", "soil_moisture_0_to_7cm", "is_day", "direct_radiation"],
            "timezone": "America/Denver"
        }
        responses = openmeteo.weather_api(url, params=params)

        # Process first location. Add a for-loop for multiple locations or weather models
        response = responses[0]
        # Process hourly data. The order of variables needs to be the same as requested.
        hourly = response.Hourly()
        df_dropped.at[i + batch_start,'dew_point_2m'] = hourly.Variables(0).ValuesAsNumpy()[hour.hour + 18]
        df_dropped.at[i + batch_start,'precipitation'] = hourly.Variables(1).ValuesAsNumpy()[hour.hour + 18]
        df_dropped.at[i + batch_start,'rain'] = hourly.Variables(2).ValuesAsNumpy()[hour.hour + 18]
        df_dropped.at[i + batch_start,'snowfall'] = hourly.Variables(3).ValuesAsNumpy()[hour.hour + 18]
        df_dropped.at[i + batch_start,'snow_depth'] = hourly.Variables(4).ValuesAsNumpy()[hour.hour + 18]
        df_dropped.at[i + batch_start, 'weather_code'] = hourly.Variables(5).ValuesAsNumpy()[hour.hour + 18]
        df_dropped.at[i + batch_start,'cloud_cover'] = hourly.Variables(6).ValuesAsNumpy()[hour.hour + 18]
        df_dropped.at[i + batch_start,'et0_fao_evapotranspiration'] = hourly.Variables(7).ValuesAsNumpy()[hour.hour + 18]
        df_dropped.at[i + batch_start,'vapour_pressure_deficit'] = hourly.Variables(8).ValuesAsNumpy()[hour.hour + 18]
        df_dropped.at[i + batch_start,'wind_gusts_10m'] = hourly.Variables(9).ValuesAsNumpy()[hour.hour + 18]
        df_dropped.at[i + batch_start,'soil_temperature_0_to_7cm'] = hourly.Variables(10).ValuesAsNumpy()[hour.hour + 18]
        df_dropped.at[i + batch_start,'soil_moisture_0_to_7cm'] = hourly.Variables(11).ValuesAsNumpy()[hour.hour + 18]
        df_dropped.at[i + batch_start,'direct_radiation'] = hourly.Variables(12).ValuesAsNumpy()[hour.hour + 18]

    time.sleep(61)
    batch_start += batch_size


OpenMeteoRequestsError: {'reason': 'Hourly API request limit exceeded. Please try again in the next hour.', 'error': True}

In [ ]:
df_dropped.to_csv('filename.csv', index=True)

Mounted at /content/drive
